In [81]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text, desc, union, union_all, asc
from sqlalchemy.sql import func, exists, text, distinct
import pandas as pd
import os

In [96]:
engine = create_engine('sqlite+pysqlite:///Pilots_db.sqlite')
conn=engine.connect()

In [97]:
sql = '''
create table Pilot(
    pilot_id int NOT NULL PRIMARY KEY,
    name varchar(18),
    age int,
    rank int,
    education_level int
)
'''
conn.execute(sql)

In [98]:
sql = '''
insert into Pilot( pilot_id, name, age, rank, education_level)
values
('1','А','27','1','18'),
('2','Абрам','28','2','7'),
('3','Аваз','52','2','3'),
('4','Август','36','2','2'),
('5','Авдей','26','6','20'),
('6','Автандил','43','2','13'),
('7','Адам','40','3','20'),
('8','Адис','27','2','15'),
('9','Адольф','41','5','5'),
('10','Адриан','26','2','5'),
('11','Азарий','48','5','4'),
('12','Аким','46','7','5'),
('13','Алан (Ален)','32','2','1'),
('14','Александр','30','5','19'),
('15','Алексей','42','5','5'),
('16','Альберт','40','2','10'),
('17','Альфред','38','2','16'),
('18','Амадей','53','3','15'),
('19','Амадеус','37','5','12'),
('20','Амаяк','51','5','16'),
('21','Анатолий','39','7','5'),
('22','Ангел','38','6','6'),
('23','Андоим','41','4','2'),
('24','Андрей','29','2','14'),
('25','Аникита','51','3','13'),
('26','Антон','48','3','17'),
('27','Ануфрий (Онуфрий)','51','4','3'),
('28','Арам','36','2','13'),
('29','Арий','35','4','10'),
('30','Аристарх','40','1','16'),
('31','Аркадий','27','5','9'),
('32','Арно','40','2','9'),
('33','Арнольд','39','1','9'),
('34','Арон','54','7','19'),
('35','Арсен, Арсений','41','5','10'),
('36','Артем','39','5','19'),
('37','Артемий','50','1','18'),
('38','Артур','38','4','9'),
('39','Архип','37','6','3'),
('40','Аскольд','36','3','13'),
('41','Афанасий','52','1','6'),
('42','Ахмет','39','4','18'),
('43','Ашот','25','5','9'),
('44','Б','48','1','20'),
('45','Бег','50','1','15'),
('46','Бежен (Бажен)','46','4','8'),
('47','Бенедикт','36','2','15'),
('48','Берек','33','7','2'),
('49','Бернар','25','4','11'),
('50','Богдан','31','5','2'),
('51','Боголюб','44','6','14'),
('52','Болеслав','29','5','15'),
('53','Бонифаций','31','5','12'),
('54','Борис','48','4','19'),
('55','Борислав','42','5','5'),
('56','Боян','38','6','12'),
('57','Бронислав','26','4','18'),
('58','Бруно','43','4','13'),
('59','В','35','6','14')
'''
conn.execute(sql)

In [99]:
sql = '''
create table Plane(
    plane_id int NOT NULL PRIMARY KEY,
    capacity int,
    cargo_flg int CHECK (cargo_flg=0 or cargo_flg=1)
)
'''
conn.execute(sql)

In [100]:
sql = '''
insert into Plane( plane_id, capacity, cargo_flg)
values
(1,232,0),
(2,290,0),
(3,254,1),
(4,115,1),
(5,167,1),
(6,339,1),
(7,126,0),
(8,164,0),
(9,117,0),
(10,67,0),
(11,93,0),
(12,257,1),
(13,329,1),
(14,177,0),
(15,261,0),
(16,204,0),
(17,87,1),
(18,322,1),
(19,172,1),
(20,109,0),
(21,62,1),
(22,230,0),
(23,58,1),
(24,260,0),
(25,33,1),
(26,61,1),
(27,86,1),
(28,214,0),
(29,208,0),
(30,244,0)
'''
conn.execute(sql)

In [101]:
sql = '''
create table Flight(
    flight_id int NOT NULL PRIMARY KEY,
    flight_dt date,
    plane_id int NOT NULL,
    first_pilot_id int NOT NULL,
    second_pilot_id int NOT NULL,
    destination varchar(18),
    quantity int
)
'''
conn.execute(sql)

In [102]:
sql = '''

insert into Flight(flight_id, flight_dt, plane_id, first_pilot_id, second_pilot_id, destination, quantity)
values
(1,'31.05.2021',14,3,59,'Шереметьево',150),
(2,'15.08.2021',3,12,30,'Шереметьево',225),
(3,'12.06.2021',26,19,40,'Домодедово',55),
(4,'10.12.2020',5,23,30,'Шереметьево',165),
(5,'18.08.2021',14,12,44,'Внуково',120),
(6,'21.08.2021',22,10,30,'Шереметьево',220),
(7,'12.08.2021',26,19,30,'Шереметьево',55),
(8,'19.08.2021',14,12,44,'Внуково',120),
(9,'15.08.2021',14,12,44,'Внуково',120),
(10,'25.08.2020',14,12,44,'Шереметьево',120),
(11,'21.05.2021',17,20,10,'Шереметьево',80),
(12,'06.12.2020',18,24,35,'Домодедово',310),
(13,'03.12.2020',12,20,56,'Шереметьево',255),
(14,'06.10.2021',3,16,3,'Внуково',210),
(15,'01.04.2021',18,47,46,'Шереметьево',250),
(16,'15.05.2021',15,32,57,'Шереметьево',200),
(17,'07.05.2021',1,47,4,'Домодедово',231),
(18,'23.05.2021',6,41,38,'Внуково',320),
(19,'22.04.2021',14,51,20,'Шереметьево',140),
(20,'15.04.2021',1,1,49,'Шереметьево',210),
(21,'29.10.2021',11,11,31,'Внуково',80),
(22,'26.12.2020',28,17,20,'Шереметьево',200),
(23,'26.01.2021',27,37,14,'Шереметьево',70),
(24,'07.04.2021',8,12,3,'Домодедово',150),
(25,'09.03.2021',29,54,34,'Шереметьево',200),
(26,'12.02.2021',30,1,49,'Шереметьево',240),
(27,'08.06.2021',10,14,16,'Домодедово',60),
(28,'08.08.2021',1,20,9,'Внуково',230),
(29,'16.11.2020',7,11,19,'Шереметьево',120),
(30,'22.04.2021',1,39,46,'Шереметьево',220),
(31,'01.04.2021',17,26,55,'Домодедово',55),
(32,'02.12.2020',9,8,36,'Шереметьево',100),
(33,'13.06.2021',30,4,58,'Шереметьево',244),
(34,'20.05.2021',23,20,49,'Домодедово',27),
(35,'20.03.2021',2,18,26,'Внуково',288),
(36,'24.07.2021',13,59,57,'Шереметьево',320),
(37,'01.09.2021',5,34,15,'Шереметьево',160),
(38,'19.02.2021',15,45,7,'Внуково',255),
(39,'03.02.2021',14,17,57,'Шереметьево',150),
(40,'22.03.2021',9,58,49,'Домодедово',90),
(41,'01.09.2021',24,18,2,'Шереметьево',220),
(42,'02.12.2020',23,41,31,'Внуково',37),
(43,'19.12.2020',27,59,39,'Шереметьево',80),
(44,'08.12.2020',19,15,22,'Шереметьево',150),
(45,'21.10.2021',18,5,29,'Внуково',321),
(46,'27.12.2020',30,26,48,'Шереметьево',220),
(47,'26.01.2021',10,25,51,'Домодедово',29),
(48,'03.06.2021',11,6,27,'Внуково',70),
(49,'29.04.2021',17,42,3,'Шереметьево',65),
(50,'15.06.2021',10,3,59,'Шереметьево',65)
'''
conn.execute(sql)